In [68]:
!pip install kaggle

In [69]:
from google.colab import files

uploaded = files.upload()

!mkdir '/root/.kaggle'
!cp 'kaggle.json' '/root/.kaggle'
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [70]:
!kaggle datasets list -s credit

ref                                                        title                                           size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  ----------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
sakshigoyal7/credit-card-customers                         Credit Card customers                         387771  2020-11-19 07:38:44.677000         134330       2429  1.0              
mlg-ulb/creditcardfraud                                    Credit Card Fraud Detection                 69155672  2018-03-23 01:17:27.913000        1052656      12813  0.85294116       
rikdifos/credit-card-approval-prediction                   Credit Card Approval Prediction              5578875  2020-03-24 10:04:48.753000          96158        913  1.0              
parisrohan/credit-score-classification                     Credit score cla

In [71]:
!kaggle datasets download -d rikdifos/credit-card-approval-prediction

Dataset URL: https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction
License(s): CC0-1.0
credit-card-approval-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)


In [72]:
!unzip "credit-card-approval-prediction.zip" -d "credit-card-datasets"

Archive:  credit-card-approval-prediction.zip
replace credit-card-datasets/application_record.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: credit-card-datasets/application_record.csv  
replace credit-card-datasets/credit_record.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: credit-card-datasets/credit_record.csv  


In [196]:
import pandas as pd

In [197]:
df_app = pd.read_csv("credit-card-datasets/application_record.csv")
df_rec = pd.read_csv("credit-card-datasets/credit_record.csv")

In [198]:
df_app.head(10)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
5,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
6,5008811,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
7,5008812,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,NaN,1.0
8,5008813,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,NaN,1.0
9,5008814,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,NaN,1.0


In [199]:
df_rec.head(10)

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
5,5001712,-1,C
6,5001712,-2,C
7,5001712,-3,C
8,5001712,-4,C
9,5001712,-5,C


In [200]:
df_app.isnull().sum()

,0
ID,0
CODE_GENDER,0
FLAG_OWN_CAR,0
FLAG_OWN_REALTY,0
CNT_CHILDREN,0
AMT_INCOME_TOTAL,0
NAME_INCOME_TYPE,0
NAME_EDUCATION_TYPE,0
NAME_FAMILY_STATUS,0
NAME_HOUSING_TYPE,0


In [201]:
df_rec["STATUS"].value_counts()

,count
STATUS,
C,442031
0,383120
X,209230
1,11090
5,1693
2,868
3,320
4,223


In [202]:
df_rec["Target"] = df_rec["STATUS"].isin(['X','C','0'])
print(df_rec["ID"].nunique(), df_rec["ID"].shape[0])
# Duplicate id is in ID columns

45985 1048575


In [203]:
df_rec_unique_id = df_rec.sort_values(["ID", "MONTHS_BALANCE"], ascending=False).drop_duplicates(subset=["ID"], keep="first")
print(df_rec_unique_id["ID"].nunique(), df_rec_unique_id["ID"].shape[0])

45985 45985


In [204]:
df_data = pd.merge(df_app, df_rec_unique_id, on="ID", how="inner").drop(columns=["MONTHS_BALANCE","STATUS", "ID"])

In [205]:
df_data.shape[0]

36457

In [206]:
df_data["Target"].value_counts()

,count
Target,
True,36075
False,382


In [207]:
df_data["Target"] = df_data["Target"].apply(lambda x: 1 if x else 0)

In [208]:
df_data.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,Target
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
2,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,1
3,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,1
4,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,1


In [209]:
df_data.isnull().sum()

,0
CODE_GENDER,0
FLAG_OWN_CAR,0
FLAG_OWN_REALTY,0
CNT_CHILDREN,0
AMT_INCOME_TOTAL,0
NAME_INCOME_TYPE,0
NAME_EDUCATION_TYPE,0
NAME_FAMILY_STATUS,0
NAME_HOUSING_TYPE,0
DAYS_BIRTH,0


In [210]:
df_data["OCCUPATION_TYPE"].value_counts()

,count
OCCUPATION_TYPE,
Laborers,6211
Core staff,3591
Sales staff,3485
Managers,3012
Drivers,2138
High skill tech staff,1383
Accountants,1241
Medicine staff,1207
Cooking staff,655


In [211]:
occupation_mode = df_data["OCCUPATION_TYPE"].mode().values[0]
df_data["OCCUPATION_TYPE"] = df_data["OCCUPATION_TYPE"].fillna(occupation_mode)
df_data["OCCUPATION_TYPE"].isnull().sum()

np.int64(0)

In [212]:
df_data.isnull().sum()

,0
CODE_GENDER,0
FLAG_OWN_CAR,0
FLAG_OWN_REALTY,0
CNT_CHILDREN,0
AMT_INCOME_TOTAL,0
NAME_INCOME_TYPE,0
NAME_EDUCATION_TYPE,0
NAME_FAMILY_STATUS,0
NAME_HOUSING_TYPE,0
DAYS_BIRTH,0


In [213]:
y = df_data.pop("Target")
x = df_data

In [214]:
x

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Laborers,2.0
1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Laborers,2.0
2,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,M,Y,Y,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,-17348,-2420,1,0,0,0,Managers,2.0
36453,F,N,Y,0,157500.0,Commercial associate,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0
36454,F,N,Y,0,157500.0,Pensioner,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0
36455,F,N,Y,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-17958,-655,1,0,0,0,Sales staff,2.0


In [215]:
print(y.shape[0] == x.shape[0])

True


In [216]:
from sklearn.model_selection import train_test_split

In [217]:
RAND_STATE=123

In [313]:
x_train, x_test, y_train, y_test = train_test_split(x,y,stratify=y, random_state=RAND_STATE, test_size=0.3)

In [314]:
print(y_train.sum() / len(y_train))
print(y_test.sum() / len(y_test))

0.9895372075708295
0.9894861949168038


In [315]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25519 entries, 1629 to 25188
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CODE_GENDER          25519 non-null  object 
 1   FLAG_OWN_CAR         25519 non-null  object 
 2   FLAG_OWN_REALTY      25519 non-null  object 
 3   CNT_CHILDREN         25519 non-null  int64  
 4   AMT_INCOME_TOTAL     25519 non-null  float64
 5   NAME_INCOME_TYPE     25519 non-null  object 
 6   NAME_EDUCATION_TYPE  25519 non-null  object 
 7   NAME_FAMILY_STATUS   25519 non-null  object 
 8   NAME_HOUSING_TYPE    25519 non-null  object 
 9   DAYS_BIRTH           25519 non-null  int64  
 10  DAYS_EMPLOYED        25519 non-null  int64  
 11  FLAG_MOBIL           25519 non-null  int64  
 12  FLAG_WORK_PHONE      25519 non-null  int64  
 13  FLAG_PHONE           25519 non-null  int64  
 14  FLAG_EMAIL           25519 non-null  int64  
 15  OCCUPATION_TYPE      25519 non-null  o

In [316]:
x_train.describe(include='all')

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
count,25519,25519,25519,25519.000000,2.551900e+04,25519,25519,25519,25519,25519.000000,25519.000000,25519.0,25519.000000,25519.000000,25519.000000,25519,25519.000000
unique,2,2,2,NaN,NaN,5,5,5,6,NaN,NaN,NaN,NaN,NaN,NaN,18,NaN
top,F,N,Y,NaN,NaN,Working,Secondary / secondary special,Married,House / apartment,NaN,NaN,NaN,NaN,NaN,NaN,Laborers,NaN
freq,17091,15867,17165,NaN,NaN,13220,17403,17527,22794,NaN,NaN,NaN,NaN,NaN,NaN,12266,NaN
mean,NaN,NaN,NaN,0.432501,1.865786e+05,NaN,NaN,NaN,NaN,-15968.554019,59009.221874,1.0,0.226380,0.294565,0.089855,NaN,2.200635
std,NaN,NaN,NaN,0.746691,1.021702e+05,NaN,NaN,NaN,NaN,4193.949230,137431.633739,0.0,0.418496,0.455856,0.285979,NaN,0.916336
min,NaN,NaN,NaN,0.000000,2.700000e+04,NaN,NaN,NaN,NaN,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,NaN,1.000000
25%,NaN,NaN,NaN,0.000000,1.215000e+05,NaN,NaN,NaN,NaN,-19420.000000,-3154.000000,1.0,0.000000,0.000000,0.000000,NaN,2.000000
50%,NaN,NaN,NaN,0.000000,1.575000e+05,NaN,NaN,NaN,NaN,-15565.000000,-1542.000000,1.0,0.000000,0.000000,0.000000,NaN,2.000000
75%,NaN,NaN,NaN,1.000000,2.250000e+05,NaN,NaN,NaN,NaN,-12461.000000,-410.000000,1.0,0.000000,1.000000,0.000000,NaN,3.000000


In [317]:
x_train["DAYS_EMPLOYED"] =  x_train["DAYS_EMPLOYED"].apply(lambda x: 0 if x == 365243 else x)

In [318]:
x_train.describe()

,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS
count,25519.000000,2.551900e+04,25519.000000,25519.000000,25519.0,25519.000000,25519.000000,25519.000000,25519.000000
mean,0.432501,1.865786e+05,-15968.554019,-2205.728203,1.0,0.226380,0.294565,0.089855,2.200635
std,0.746691,1.021702e+05,4193.949230,2367.500245,0.0,0.418496,0.455856,0.285979,0.916336
min,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000
25%,0.000000,1.215000e+05,-19420.000000,-3154.000000,1.0,0.000000,0.000000,0.000000,2.000000
50%,0.000000,1.575000e+05,-15565.000000,-1542.000000,1.0,0.000000,0.000000,0.000000,2.000000
75%,1.000000,2.250000e+05,-12461.000000,-410.000000,1.0,0.000000,1.000000,0.000000,3.000000
max,19.000000,1.575000e+06,-7489.000000,0.000000,1.0,1.000000,1.000000,1.000000,20.000000


In [319]:
mapping_education = [
    'Lower secondary',
    'Secondary / secondary special',
    'Incomplete higher',
    'Higher education',
    'Academic degree'
]
# x_train["NAME_EDUCATION_TYPE"] = x_train["NAME_EDUCATION_TYPE"].map(mapping_education)

In [320]:
cat_feature = x_train.select_dtypes(include='object').columns.tolist()
num_feature = x_train.select_dtypes(exclude='number').columns.tolist()
cat_feature.remove("NAME_EDUCATION_TYPE")

In [321]:
print(cat_feature)

['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE']


In [322]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

one_hot_enc = OneHotEncoder()
one_hot_enc.fit(x_train[cat_feature])
ordinaL_enc = OrdinalEncoder(categories=[mapping_education])
ordinaL_enc.fit(x_train[["NAME_EDUCATION_TYPE"]])

OrdinalEncoder(categories=[['Lower secondary', 'Secondary / secondary special',
                            'Incomplete higher', 'Higher education',
                            'Academic degree']])

In [323]:
encoded_one_hot = one_hot_enc.transform(x_train[cat_feature])
encoded_ordinal = ordinaL_enc.transform(x_train[["NAME_EDUCATION_TYPE"]])
print(len(cat_feature),encoded_one_hot.shape[1])
print(encoded_ordinal)

7 40
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [324]:
new_cat_cols = []
for i in range(len(cat_feature)):
  for cal_val in one_hot_enc.categories_[i]:
    new_cat_cols.append(cat_feature[i]+"_"+cal_val)

In [325]:
new_cat_cols

['CODE_GENDER_F',
 'CODE_GENDER_M',
 'FLAG_OWN_CAR_N',
 'FLAG_OWN_CAR_Y',
 'FLAG_OWN_REALTY_N',
 'FLAG_OWN_REALTY_Y',
 'NAME_INCOME_TYPE_Commercial associate',
 'NAME_INCOME_TYPE_Pensioner',
 'NAME_INCOME_TYPE_State servant',
 'NAME_INCOME_TYPE_Student',
 'NAME_INCOME_TYPE_Working',
 'NAME_FAMILY_STATUS_Civil marriage',
 'NAME_FAMILY_STATUS_Married',
 'NAME_FAMILY_STATUS_Separated',
 'NAME_FAMILY_STATUS_Single / not married',
 'NAME_FAMILY_STATUS_Widow',
 'NAME_HOUSING_TYPE_Co-op apartment',
 'NAME_HOUSING_TYPE_House / apartment',
 'NAME_HOUSING_TYPE_Municipal apartment',
 'NAME_HOUSING_TYPE_Office apartment',
 'NAME_HOUSING_TYPE_Rented apartment',
 'NAME_HOUSING_TYPE_With parents',
 'OCCUPATION_TYPE_Accountants',
 'OCCUPATION_TYPE_Cleaning staff',
 'OCCUPATION_TYPE_Cooking staff',
 'OCCUPATION_TYPE_Core staff',
 'OCCUPATION_TYPE_Drivers',
 'OCCUPATION_TYPE_HR staff',
 'OCCUPATION_TYPE_High skill tech staff',
 'OCCUPATION_TYPE_IT staff',
 'OCCUPATION_TYPE_Laborers',
 'OCCUPATION_TYPE_L

In [326]:
x_train_without_education = pd.DataFrame(encoded.toarray(), columns=new_cat_cols)
x_train_with_education = pd.DataFrame(encoded_ordinal, columns=["NAME_EDUCATION_TYPE"])
x_train_transform = pd.concat([x_train_without_education, x_train_with_education], axis=1)

In [327]:
x_train_transform.head()

,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,...,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,NAME_EDUCATION_TYPE
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0
4,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [330]:
x_train["DAYS_EMPLOYED"]

,DAYS_EMPLOYED
1629,-5578
17586,-10490
34388,-5184
18588,-232
10459,-1539
...,...
32745,-3092
9980,-2351
13037,-110
10444,0


In [331]:
x_train["DAYS_BIRTH"] = (x_train["DAYS_BIRTH"] / -365).astype(int)
x_train["DAYS_EMPLOYED"] = round(x_train["DAYS_EMPLOYED"] / -365,2)

In [332]:
x_train["DAYS_BIRTH"]

,DAYS_BIRTH
1629,47
17586,54
34388,47
18588,25
10459,29
...,...
32745,32
9980,43
13037,56
10444,58


In [333]:
x_train["DAYS_EMPLOYED"]

,DAYS_EMPLOYED
1629,15.28
17586,28.74
34388,14.20
18588,0.64
10459,4.22
...,...
32745,8.47
9980,6.44
13037,0.30
10444,-0.00
